Nachfragemodellierung
- Gesamtbevölkerung 84,37 Mil: https://de.statista.com/statistik/daten/studie/2861/umfrage/entwicklung-der-gesamtbevoelkerung-deutschlands/
- BEvölkerung Sachsen 4,09 Millionen Einwohner: https://de.statista.com/statistik/daten/studie/155167/umfrage/entwicklung-der-bevoelkerung-von-sachsen-seit-1961/

Quelle: https://milchindustrie.de/wp-content/uploads/2023/09/ZahlenDatenFakten_2023_neu.pdf

| | Milchpulver | Konsummilch | Butter | Käse |
| --- | --- | --- | --- | --- |
| Herstellungsmenge (T Tonnen) | 630 | 7300 | 480 | 2670 |

In [6]:
anteil = 4.09 / 84.37
anteil_jahr = 30 / 360
products = [630, 7300, 480, 2670]
products_right = [int(round(prod * 1000 * anteil * anteil_jahr,0)) for prod in products]

products_right

[2545, 29490, 1939, 10786]

In [ ]:
#1,7 Milliarden kg  Sachsenmilch Webiste jährlich prod menge 



In [25]:
#milchanlieferung = round((31021.6 * 1000) * (30/360) * anteil)
milchanlieferung = round((1.7 * 10**6) * (30/360))

In [26]:
milchanlieferung

141667

In [1]:
import pandas as pd

# Gegebene Liste mit Sublisten
values_list = [
    [6400, 8000, 12000],
    [600, 1000, 1200],
    [2800, 4000, 8000],
    [300, 500, 600],
    [862.5, 1725, 3450]
]

# Berechnung der prozentualen Schwankungen
percent_changes = []

for sublist in values_list:
    original_value = sublist[1]  # Das mittlere Produkt entspricht dem Original
    changes = []
    for value in sublist:
        percent_change = ((value - original_value) / original_value) * 100
        changes.append(percent_change)
    percent_changes.append(changes)

print(percent_changes)

# Erstellen eines DataFrames für eine bessere Darstellung
#df_changes = pd.DataFrame(percent_changes, columns=["Schwankung 1 (%)", "Original (%)", "Schwankung 3 (%)"])

# DataFrame anzeigen
#print(df_changes)


[[-20.0, 0.0, 50.0], [-40.0, 0.0, 20.0], [-30.0, 0.0, 100.0], [-40.0, 0.0, 20.0], [-50.0, 0.0, 100.0]]


In [15]:
products_right.append(milchanlieferung)

In [20]:
products_right

[2545, 29490, 1939, 10786, 125319]

In [24]:
new_values = []

for p in range(len(products_right)):
    sublist = []
    for change in percent_changes[p]:
        sublist.append(round(products_right[p] * (1 + change / 100)))

    new_values.append(sublist)

new_values

[[2036, 2545, 3818],
 [17694, 29490, 35388],
 [1357, 1939, 3878],
 [6472, 10786, 12943],
 [62660, 125319, 250638]]

{
    "T_No": 30,
    "F_No": 4,
    "FT_No": 2,
    "MP_No": 4,
    "CT_No": 2,
    "L_No": 9,
    "fty": [0,0,1,1],
    "cty": [0,1,1,1],
    "fpr": [0,1,2,3],
    "fy": [0.12,0.975, 0.9, 0.11],
    "rsc": 6,
    "roc": 12,
    "el": [25,25,25,25],
    "tau": [2,1,1,2,0,1,0,2,1],
    "i_0": [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
    "i_0_f": [0,0,0,0],
    "tl_min": 4,
    "tl_max": 8,
    "r0": 0,
    "r_max": 200,
    "dmax": [15,100,100,100],
    "cmin": [1, 1, 1, 1],
    "cmax": [120,330,450,51],
    "alpha": [2,0,0,0],
    "ost": [0, 0, 0, 0],
    "wp": [[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [20, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[10,20,15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
    "el_min": [0,0,0,0],
    "el_max": [120,0,0,0],
    "is": [0, 0.95, 0.95, 0.95], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "omega_fw": [40, 40, 5, 10],
    "omega_dc": [100, 100, 7, 14],
    "rr": [3.75, 1, 0, 0], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "r": [12.5, 20,20, 12], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "re": [5, 0, 0, 0], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "imax": [[12,10], [68, 114], [16, 35], [16,40],[9, 27],[31,54],[20, 81],[20,58],[58,195]], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "zmax": [1,3,3,3], 
    "sc": [0,110,150,17],NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "beta": [0.1,0,0,0], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "sigma": [1,0,0,4], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "iwip0": [0, 0, 0, 0],
    "tc": [[15.641,13.034], [4.956, 4.13], [10.332,8.61], [14.818, 12.348], [0.067, 0.056],[14.364, 11.97], [4.032, 3.36], [21.638,18.032],[9.021,7.518]], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "sco": 20, NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 
    "K": 1,
    "epsilon": 0.001,
    "N": 10000,
    "demand_supply": [[6400, 8000, 12000],[600, 1000, 1200],[2800, 4000, 8000],[300, 500, 600],[862.5, 1725, 3450]],
    "probabilies": [[0.4, 0.35, 0.25], [0.25, 0.35, 0.4], [0.4, 0.3, 0.3], [0.3, 0.4, 0.3], [0.5, 0.15, 0.35]]
}

Festlegung: 

4 Produkltfamilien(Powdered Milk, UHT, Joghurt, Cheese)
8 Produkte (Babymilchpulver, Milchpulver, 1.5 % Milch UHT, 3.5 % Milch UHT, Griechischer Joghurt)
2 Fresh and 2 Dry! 
Jede Produltfamilie besitzt eigene Fertigungsfläche und Lagerung dort 
3 Distribution Centers 
Ein Monat Planungszeitraum 

"T_No": 30,
"F_No": 4,
"FT_No": 2,
"MP_No": 4,
"CT_No": 2,
"L_No": 9,
"P_No": 8,

Gleichlassen von: 
"fty": [0,0,1,1],
"cty": [0,1,1,1],
"fpr": [0,1,2,3],

Bitte realistische Werte nachschauen: --> Wert unter 0.1 macht keinen SINN !! 
    "fy": [0.12,0.975, 0.9, 0.11],

QUELLE: 
QUELLE: 

Bitte realistische Werte für Kosten und 

"rsc": 6,
"roc": 12,

QUELLE: 
QUELLE: 

Fahrzeiten 1 Tag unterschied für jeden Transport 
"tau": [1,1,1],

 Anfangsbestände unberührt lassen !!    
    "i_0": [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],
    "i_0_f": [0,0,0,0],
    "r0": 0,
     "wp": [[100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [20, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[10,20,15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

Export eigentlich unbegrenzt, da Nachbarländer --> Annahme, dass Preise günstiger:

"el_min": [0,0,0,0],
"el_max": [1000,1000,1000,1000],

"is": [0, 0.95, 0.95, 0.95], NACHSCHAUEN + QUELLEN ORDENTLICH BEARBEITEN 

Der Preis für ein Kilogramm Milch lag in Deutschland im April 2024 bei durchschnittlich rund 45,9 Cent, den die abnehmenden Molkereien an die Milchbauern zahlten. Je nach Geschäftsstruktur, Region und Jahreszeit zahlen die Molkereien jedoch individuelle Preise an die Landwirte aus, sodass im Milchpreis Unterschiede entstehen können. Die Abrechnung von Milch in der Landwirtschaft erfolgt stets in Kilogramm, doch der Lebensmittelhandel bietet Milch meist in Literverpackungen an – ein Liter Milch entspricht etwa 1,03 Kilogramm.
https://de.statista.com/statistik/daten/studie/380546/umfrage/preis-von-kuhmilch-in-deutschland/

Produktion jährlich rund 1,7 Milliarden kg --> 1,7 Millionenn Tonnen Milch: Quelle; https://www.sachsenmilch.de/ueber-uns/unsere-molkerei

Nachfrage nach Produkten: 
Ein Bundesbürger verzehrt im Jahr...
... 49,5 Kilogramm Konsummilch ... 25,1 Kilogramm Käse
... 15,1 Kilogramm Joghurt
... 5,8 Kilogramm Butter


155 milchverarbeitende Unternehmen mit 39.131 Beschäftigen*...
... erwirtschaften einen Umsatz von 27,0 Mrd. Euro...
... und verarbeiten 31,7 Millionen Tonnen Milch pro Jahr zu... ... 4,5 Millionen Tonnen Konsummilch
... 2,4 Millionen Tonnen Käse1)
... 497.100 Tonnen Butter2)
... 554.400 Tonnen Sahne
... 392.800 Tonnen Magermilchpulver

Quelle: 
https://milchindustrie.de/wp-content/uploads/2020/09/Fakten-Milch-September-2020.pdf
https://milchindustrie.de/wp-content/uploads/2023/09/ZahlenDatenFakten_2023_neu.pdf
